## Artificial Neural Network

## Imprting the LBs

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
tf.__version__

'2.15.0'

## Importing the dataset

In [6]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

print(X)

In [8]:
print(y)

[1 0 1 ... 1 1 0]


## Encoding categorical data

### Label Encoding the "Gender" column

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [10]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


### One Hot Encoding the "Geography" column

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers =[('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

In [12]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


## Splitting the dataset into the Training set and Test set 

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Feautre Scaling

In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Building the ANN

### Initializing the ANN

In [15]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [16]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [17]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) 

### Adding the output layer

In [18]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training the ANN

### Compiling the ANN

모델을 컴파일한다는 것은 모델이 학습을 위해 사용할 옵티마이저, 손실 함수, 평가 지표를 지정하는 과정을 말합니다.

In [22]:
# Adam은 경사 하강법의 한 종류로, 학습 속도를 조절하면서 최적의 매개변수를 찾아가는 알고리즘
# 'binary_crossentropy'는 이진 분류(binary classification) 문제에서 사용하는 손실 함수 중 하나
# 'accuracy' : 모델의 정확도를 평가하는 지표입니다. 정확도는 모델이 올바르게 분류한 샘플의 비율을 나타냅니다.
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy' , metrics = ['accuracy'])

### Training the ANN on the Training set

In [23]:
# 배치 크기는 한 번의 반복(에폭)에서 사용할 샘플의 개수를 의미
# 에폭은 전체 훈련 데이터셋을 한 번 모델에 보여준 것을 말합니다. 
# 따라서 이 매개변수는 전체 훈련 데이터셋을 몇 번 반복하여 모델을 훈련할지를 결정합니다.
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 768us/step - loss: 0.5574 - accuracy: 0.7581
Epoch 2/100
250/250 [==============================] - 0s 583us/step - loss: 0.4777 - accuracy: 0.7959
Epoch 3/100
250/250 [==============================] - 0s 504us/step - loss: 0.4575 - accuracy: 0.7975
Epoch 4/100
250/250 [==============================] - 0s 531us/step - loss: 0.4442 - accuracy: 0.8016
Epoch 5/100
250/250 [==============================] - 0s 612us/step - loss: 0.4368 - accuracy: 0.8023
Epoch 6/100
250/250 [==============================] - 0s 639us/step - loss: 0.4321 - accuracy: 0.8051
Epoch 7/100
250/250 [==============================] - 0s 572us/step - loss: 0.4285 - accuracy: 0.8067
Epoch 8/100
250/250 [==============================] - 0s 526us/step - loss: 0.4248 - accuracy: 0.8069
Epoch 9/100
250/250 [==============================] - 0s 695us/step - loss: 0.4192 - accuracy: 0.8075
Epoch 10/100
250/250 [==============================] - 0s 635us/step - l

## Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

In [26]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 28ms/step
[[False]]


## Predicting the Test set results

In [29]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test), 1)), 1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


## Making the Confusion Matrix

In [30]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1536   59]
 [ 212  193]]


0.8645